In [3]:
#how = 'Instances '
#how = 'Types '
#how = 'Class '

#how = 'Event '
how = 'Instances '
experiment_name = 'reborn_events_community' 
method = 'community'



 #range, event, baseline instances, baseline class
#experiment_name = '' and here you put baseline instance, baseline class etc etc 
#[event1_relation_all, events12_community_types, events12_community_range, events12_community_all]
#for event1_relation_all need old version

In [4]:
### import libraries

import transformers
from datasets import load_dataset, load_metric

import numpy as np
import os
import nltk
import torch
import evaluate
nltk.download('punkt')
print(transformers.__version__)
print(torch.__version__)

4.24.0
2.0.0.dev20230105


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/macoftraopia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
from nltk.tokenize import word_tokenize
w = '''Once upon a time, in The Vale of Arryn there was a procurer whose name was Cersei Lannister. Cersei Lannister was a Lady from the House Targaryen. It was sunrise in Red Keep when Arya Stark threatened the  castle of Cersei Lannister . Cersei Lannister was scared of the teleportation power of Arya Stark and refused to fight back. It was Christmas in Kings Landing when Cersei Lannister met Tommen Baratheon .  From Tommen Baratheon Cersei Lannister learnt the power of fire Resistance. It was the first day of Spring when Cersei Lannister finally gatherered the  courage to travel to White Harbor.  Cersei Lannister met Shae. Together they formed an alliance and fought against Eddard Stark. It resulted in an epic Loss. It was Christmas in The Ruby Ford when  Cersei Lannister faced Arya Stark. Cersei Lannister used the power of fire Resistance , but Arya Stark used pausing Time. Our hero was Injured. After a bloody fight, Cersei Lannister finally saved freedom. Soon after Arya Stark was hungry for revenge and chased down Cersei Lannister. It was the first day of Spring in The Twins,  when our hero met Arya Stark again, and they fought in the final battle. Cersei Lannister won the duel and took the life of the villain. Finally it was Easter day in Red Keep when Cersei Lannister was finally safe, and celebrated with Shae by listening to a storytelling. '''
s = word_tokenize(w)
l = ['1','r']
counter = 0
for item in s:
    # Incrementing counter variable to get each item in the list
    counter = counter + 1
print(counter)    

256


In [6]:

Working_Dir = '/Users/macoftraopia/Documents/GitHub/GoTAutomyth_short'




In [7]:
### upload the data


train_file = Working_Dir + '/generated_output/' + experiment_name + '/' + method +'_train.json'
dev_file = Working_Dir + '/generated_output/' +  experiment_name + '/' + method + '_val.json'
test_file = Working_Dir + '/generated_output/' + experiment_name + '/' + method + '_test.json'

dataset = load_dataset('json', data_files={'train': train_file, 'valid': dev_file, 'test': test_file})

### let's check the data

print(len(dataset['valid'][0]['story']))
print(dataset['test'][0][f'{how}Knowledge Graph'])

Using custom data configuration default-e9ac48d5d2322d02
Found cached dataset json (/Users/macoftraopia/.cache/huggingface/datasets/json/default-e9ac48d5d2322d02/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)
100%|██████████| 3/3 [00:00<00:00, 385.21it/s]

807
Tyrion Lannister - Threatens - The House | Daenerys Targaryen - meetsMentor - Tommen Baratheon | Final confrontation between hero and villain - Time5 - sunrise | Daenerys Targaryen - fights - Tyrion Lannister | The hero meets ally - Place4 - Seagard | Daenerys Targaryen - celebratesvictory - organizing a concert | Daenerys Targaryen - hasHouse - House Targaryen | Daenerys Targaryen - meetsAlly - Rhaenys Targaryen | Daenerys Targaryen - hasTitle - Princess | Daenerys Targaryen - powerLearned - military strategy | Daenerys Targaryen - usespower - military strategy | Daenerys Targaryen - saves - The House | The hero meets ally - Time4 - sunrise | Daenerys Targaryen - helpedby - Rhaenys Targaryen | Final confrontation between hero and villain - Place5 - The Ruby Ford | Daenerys Targaryen - partywith - Rhaenys Targaryen | Daenerys Targaryen - hasOccupation - a ruler | 


In [8]:
print(dataset['test'][0][f'{how}Knowledge Graph'])

Tyrion Lannister - Threatens - The House | Daenerys Targaryen - meetsMentor - Tommen Baratheon | Final confrontation between hero and villain - Time5 - sunrise | Daenerys Targaryen - fights - Tyrion Lannister | The hero meets ally - Place4 - Seagard | Daenerys Targaryen - celebratesvictory - organizing a concert | Daenerys Targaryen - hasHouse - House Targaryen | Daenerys Targaryen - meetsAlly - Rhaenys Targaryen | Daenerys Targaryen - hasTitle - Princess | Daenerys Targaryen - powerLearned - military strategy | Daenerys Targaryen - usespower - military strategy | Daenerys Targaryen - saves - The House | The hero meets ally - Time4 - sunrise | Daenerys Targaryen - helpedby - Rhaenys Targaryen | Final confrontation between hero and villain - Place5 - The Ruby Ford | Daenerys Targaryen - partywith - Rhaenys Targaryen | Daenerys Targaryen - hasOccupation - a ruler | 


In [9]:
print(len(dataset['valid'][0]['story']))
s = (dataset['valid'][0]['story'])

807


In [10]:
### data looks OK, let's continue with the tokenizer
max_input = 1000 #number of tokens we expect -DEPENDS ON size of inputs (for the tensor) - sentence and then it pads the rest - 
#such that tensors equal in shape (no pads when training bc we alreasy have same shape) - batches of same size
max_target = 1000
model_checkpoints = 'facebook/bart-base'
tokenizer = transformers.AutoTokenizer.from_pretrained(model_checkpoints)

In [11]:
def preprocess_data(data_to_process):
  #get the dialogue text
  inputs = [graph for graph in data_to_process[f'{how}Knowledge Graph']]
  #tokenize text
  model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)

  #tokenize labels
  #with tokenizer.as_target_tokenizer():
  targets = [target for target in data_to_process['story']]
  model_targets = tokenizer(targets, max_length=max_target, padding='max_length', truncation=True)
    
  model_inputs['labels'] = model_targets['input_ids']
  #reuturns input_ids, attention_masks, labels
  return model_inputs

In [20]:
tokenize_data = dataset.map(preprocess_data, batched = True) 

print(tokenize_data['test']['story'][3])
print(tokenize_data['test'][f'{how}Knowledge Graph'][3])

### let's drop the columns that we won't need
tokenize_data = dataset.map(preprocess_data, batched = True, remove_columns=['story','Instances Knowledge Graph','Class Knowledge Graph', 'Types Knowledge Graph', 'Event Knowledge Graph', 'Range Knowledge Graph','Ontology Knowledge Graph'])
print(tokenize_data)

  0%|          | 0/1 [00:00<?, ?ba/s]


Once upon a time, there was a regent whose name was Cersei Lannister. Cersei Lannister was a Queen regnant from the House Lannister. Everything was quiet when one day Bran Stark threatened the freedom of Cersei Lannister. Cersei Lannister did not know what to do and went to Daario Naharis to seek help. Daario Naharis was a master of invisibility and taught the hero all its secrets. On the way to Riverlands at the Autumnal Equinox, Cersei Lannister met Shae and they immediately became friends. At the first day of Spring, Cersei Lannister and Shae were almost in The Ruby Ford when they realized Bran Stark was after them. The two friends nevertheless were ready and the battle begun. Cersei Lannister used invisibility forcing the enemy to retreat. Eventually the peace was safe again. Cersei Lannister and Shae went to Castle Black and celebrated by playing a game.
Cersei Lannister - hasOccupation - a regent | Cersei Lannister - meetsMentor - Daario Naharis | Bran Stark - Threatens - the fre

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 500
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 50
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 50
    })
})


In [21]:
### load model
model = transformers.AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)
#batch_size = 4 #we dont use htis 
#collator to create batches. It preprocess data with the given tokenizer
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model)

loading configuration file config.json from cache at /Users/macoftraopia/.cache/huggingface/hub/models--facebook--bart-base/snapshots/aadd2ab0ae0c8268c7c9693540e9904811f36177/config.json
Model config BartConfig {
  "_name_or_path": "facebook/bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABE

In [14]:
model

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [22]:
#####################
# metrics
# compute rouge for evaluation 
#####################
metric = load_metric('rouge')

def compute_rouge(pred):
  predictions, labels = pred
  #decode the predictions
  decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  #decode labels
  decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  #compute results
  res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
  #get %
  res = {key: value.mid.fmeasure * 100 for key, value in res.items()}

  pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  res['gen_len'] = np.mean(pred_lens)

  return {k: round(v, 4) for k, v in res.items()}

In [28]:
args = transformers.Seq2SeqTrainingArguments(
    'output',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size= 1,
    gradient_accumulation_steps=2, #compute gradient on 2 examples KG story 
    weight_decay=0.01, #regularization
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True, #since we use validation (bc during validation we generate and compare to gold ) - backprpop error on rouge
    generation_max_length = 1000, #max number of tokens per generation 
    generation_num_beams=5, #decoding strategy! greedy search, beam search 
    eval_accumulation_steps=1, #backprop  
    fp16=False #memory management 
    )
#only CUDA available -> fp16=True

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [25]:
### almost training time
trainer = transformers.Seq2SeqTrainer(
    model, 
    args,
    train_dataset=tokenize_data['train'],
    eval_dataset=tokenize_data['valid'],
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_rouge
)

In [29]:
### training time - takes around 15 mins

trainer.train()
### save the model for later use

trainer.save_model(Working_Dir + "/tuned_models/BART_models/" + experiment_name + how)

***** Running training *****
  Num examples = 500
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 2
  Total optimization steps = 750
  Number of trainable parameters = 139420416

  6%|▌         | 44/750 [34:47<9:18:21, 47.45s/it]














































KeyboardInterrupt: 

In [ ]:
### let's have a look at the predictions

preds, labels, metrics = trainer.predict(tokenize_data['test'], num_beams=5, min_length=50, max_length=128, no_repeat_ngram_size=2, early_stopping=True)

: 

In [ ]:
pred = []
lab = []
for gen, gold in zip(preds, labels):
  gen = tokenizer.decode(gen, skip_special_tokens=True)
  gen = str(gen)
  gen = gen
  pred.append(gen)
  #print(pred)
  #print(f'Generated text: {gen}')
  gold = tokenizer.decode(gold, skip_special_tokens=True)
  gold = str(gold)
  gold = [gold]
  lab.append(gold)
  #results = bleu.compute(predictions=gen, references=gold)
  #print(results)
  #print(f'Reference text: {gold}')
print(len(pred))  
print(len(lab))

: 

In [ ]:

bleu = evaluate.load("bleu")
result_bleu= bleu.compute(predictions=pred, references=lab)
print(result_bleu)

google_bleu = evaluate.load("google_bleu")
result_google_bleu = google_bleu.compute(predictions=pred, references=lab)
print(result_google_bleu)

: 

In [ ]:
preds

: 

In [ ]:
os.mkdir(Working_Dir + '/generated_text/by_BART/' + experiment_name + how )
outfile_path = Working_Dir + '/generated_text/by_BART/' + experiment_name + how + '/generated_text_with_refs_bleu.txt'

outfile = open(outfile_path, "a", encoding='utf-8')

scores = metrics.items()
print(f'Results: {scores}')
print(f'Bleu Results: {result_bleu}')
print(f'Google Bleu Results: {result_google_bleu}')

outfile.write(f'Test Results: {scores}\n\n')
outfile.write(f'Bleu Results: {result_bleu}\n\n')
outfile.write(f'Google Bleu Results: {result_google_bleu}\n\n')

for gen, gold in zip(preds, labels):
  gen = tokenizer.decode(gen, skip_special_tokens=True)
  print(f'Generated text: {gen}')
  outfile.write(f'Generated text: {gen}\n')

  gold = tokenizer.decode(gold, skip_special_tokens=True)
  print(f'Reference text: {gold}')
  outfile.write(f'Reference text: {gold}\n\n')


  
outfile.close()

: 

In [ ]:
for i in metrics.items():
  print(i)

: 